# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [16]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [17]:
#Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [18]:
# Setting a Keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [21]:
#Creating table musicl library

query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

    

                    

In [22]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERTing the values into music_library
        query = "INSERT INTO music_library ( sessionId, itemInSession,artist,song_title,song_length) "
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        ##Here we assing the line of the csv file into our columns on our DB`
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [23]:
## Select query that finds all artist,song titles, song lenghts where session id = 338 and itemsession 4 
query = "SELECT artist,song_title,song_length,sessionId,itemInSession from music_library where sessionId =338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.sessionid,row.iteminsession,row.artist,row.song_title,row.song_length)

338 4 Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [24]:
## Creating new table artist_library

query = "CREATE TABLE IF NOT EXISTS artist_library"
query = query + "(userId int, sessionId int, artist_name text,song text,itemInSession int,firstName text,lastName text, PRIMARY KEY ( (userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


<Error from server: code=2000 [Syntax error in CQL query] message="line 1:194 mismatched input ')' expecting EOF (...userId, sessionId), itemInSession))[)])">


In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERTing the values into music_library
        query = "INSERT INTO artist_library (userId,sessionId,artist_name,song, itemInSession,firstName,lastName) "
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        ##Here we assing the line of the csv file into our columns on our DB
        session.execute(query, (int(line[10]), int(line[8]),line[0],line[9],int(line[3]),line[1],line[4]))

In [14]:
## Getting name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 73, sessionid = 152
query = "SELECT artist_name,song,firstName,lastName from artist_library where sessionId = 518 and userId = 73 Allow filtering "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song,row.firstname,row.lastname)

Erin Bode Here_ There And Everywhere Jacob Klein
Lionel Richie Hello Jacob Klein
Datarock True Stories Jacob Klein
Metallica Master Of Puppets Jacob Klein
Drake / Kanye West / Lil Wayne / Eminem Forever Jacob Klein
Vanity Fare Early In The Morning Jacob Klein
Lighter Shade of Brown Homies Jacob Klein
Emmy The Great Mia Jacob Klein
B.o.B Nothin' On You [feat. Bruno Mars] (Album Version) Jacob Klein
Bikini Kill Reject All American Jacob Klein
Jake Hess You And Me Jesus Jacob Klein
Collective Soul Reunion (LP Version) Jacob Klein
The Black Keys Every Lasting Light Jacob Klein
Che Sudaka Bihotza Jacob Klein
Aloe Blacc I Need A Dollar Jacob Klein
Big Tymers / Gotti / TQ / Mikkey Sunny Day Jacob Klein
Arctic Monkeys From The Ritz To The Rubble Jacob Klein
The Mars Volta With Twilight as My Guide Jacob Klein
Jets To Brazil Cat Heaven Jacob Klein
The Knife Silent Shout Jacob Klein
Benjy Ferree Fear Jacob Klein
Onra TAKE A RIDE Jacob Klein
The Black Crowes Mellow Down Easy Jacob Klein
Molotov F

In [ ]:
## Creating user_table

query = "CREATE TABLE IF NOT EXISTS user_table"
query = query + "(userId int, firstName text, lastName text, song text, PRIMARY KEY (song,userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)      

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERTing the values into music_library
        query = "INSERT INTO table_final ( userId,firstName,lastName,song) "
        query = query + "VALUES(%s,%s,%s,%s)"
        ##Here we assing the line of the csv file into our columns on our DB
        session.execute(query, (int(line[10]),line[1],line[4],line[9]))

In [224]:
## Getting user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "SELECT firstName,lastName from table_final where song = 'All Hands Against His Own' ALLOW FILTERING"
#query = "SELECT * from table_final"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(userid=23, firstname='Morris', lastname='Gilmore', song='Just Lose It')
Row(userid=53, firstname='Celeste', lastname='Williams', song='Fireflies')
Row(userid=91, firstname='Jayden', lastname='Bell', song='Spin Spin Sugar')
Row(userid=55, firstname='Martin', lastname='Johnson', song='Bitter Sweet Symphony')
Row(userid=33, firstname='Bronson', lastname='Harris', song='In This World')
Row(userid=5, firstname='Elijah', lastname='Davis', song='Anything You Say (Unreleased Version)')
Row(userid=28, firstname='Brantley', lastname='West', song="Day 'N' Nite")
Row(userid=42, firstname='Harper', lastname='Barrett', song='Microdaze')
Row(userid=50, firstname='Ava', lastname='Robinson', song='Unfamiliar Faces')
Row(userid=95, firstname='Sara', lastname='Johnson', song='Sunday Morning')
Row(userid=88, firstname='Mohammad', lastname='Rodriguez', song='Never Leave Me Alone')
Row(userid=82, firstname='Avery', lastname='Martinez', song='DÃ\x83Â\xadmelo')
Row(userid=49, firstname='Chloe', lastname='

### Drop the tables before closing out the sessions

In [207]:
## Dropping the three tables

In [15]:
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table table_final"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table table_final"


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()